In [1]:
import pandas as pd

res_df = pd.read_csv("../data/adns.a", header = None, sep = " ")
res_df.columns = ["fqdn", "ip"]
vt_df = pd.read_csv("../data/mal_urls.csv.scan.report.csv", header = None, sep = "##")
vt_df.columns = ["url", "timestamp", "id", "positives", "avs", "ip"]

/home/local/QCRI/mnabeel/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [2]:
res_df.head(5)

,fqdn,ip
0,d3ag4hukkh62yn.cloudfront.net,13.35.176.39
1,l-0005.dc-msedge.net,13.107.43.14
2,tatar-congress.org,185.75.47.6
3,arsitios.com,172.67.177.187
4,arsitios.com,104.31.71.181


In [3]:
vt_df.head(5)

,url,timestamp,id,positives,avs,ip
0,https://www.amazon.com/gp/product/?tag=calcula...,2021-01-13 11:11:17,313dd00dbc4db855b010891097ddbbd49dce22e91c9077...,0,NaN,13.226.21.44
1,https://www.linkedin.com/feed/update/urn:li:sh...,2021-01-13 11:11:19,385988ca361b1f14d0668ab32c982c893943ebbcf43938...,0,NaN,13.107.42.14
2,http://tatar-congress.org/ru/yanalyklar/video-...,2021-01-13 11:11:19,014c731f5479c72d619fb4f57e7cc85bc17d6c78ea4221...,0,NaN,185.75.47.6
3,https://arsitios.com/zonafrancatv/2020/08/17/v...,2021-01-13 11:11:20,d6e4b90ad2f9285c6c9af41d64b5ed68fe1884bf73ff30...,1,Kaspersky:phishing site,172.67.177.187
4,http://guiaglobal.com.br/2020/08/17/imunologis...,2021-01-13 11:11:21,10ac268d65affc488c0ca1abc26efc67e39680f66075e9...,0,NaN,191.252.123.29


In [4]:
vt_df[vt_df["ip"].notnull()].shape

(423, 6)

In [36]:
def get_subnet24(ip):
    i = ip.rindex(".")
    return ip[:i]

In [5]:
import re
def is_ipv4(ip):
    if re.match('\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ip) != None:
        return True
    return False

In [6]:
import sys
sys.path.append("../../Common/utils")
from maxmind_tools import MyASN

asn = MyASN("../data/geo_20200901_GeoIPASNum.dat")

In [7]:
vt_df["asn"] = vt_df["ip"].apply(lambda x: asn.getasn(x) if is_ipv4(x) else None)

In [8]:
from domain_tools import get_2ld, get_fqdn
vt_df["fqdn"] = vt_df["url"].apply(get_fqdn)
vt_df["apex"] = vt_df["fqdn"].apply(get_2ld)

In [9]:
vt_df["org"] = vt_df["ip"].apply(lambda x: asn.getorg(x) if is_ipv4(x) else None)

In [10]:
def getasn_dom(asn, dom):
    try:
        return asn.getasn_dom(dom)
    except:
        return None
    
vt_df["asn_dom"] = vt_df["fqdn"].apply(lambda x: getasn_dom(asn, x))

In [11]:
vt_df.head()

,url,timestamp,id,positives,avs,ip,asn,fqdn,apex,org,asn_dom
0,https://www.amazon.com/gp/product/?tag=calcula...,2021-01-13 11:11:17,313dd00dbc4db855b010891097ddbbd49dce22e91c9077...,0,NaN,13.226.21.44,AS16509,www.amazon.com,amazon.com,AMAZON-02,AS16509 AMAZON-02
1,https://www.linkedin.com/feed/update/urn:li:sh...,2021-01-13 11:11:19,385988ca361b1f14d0668ab32c982c893943ebbcf43938...,0,NaN,13.107.42.14,AS8068,www.linkedin.com,linkedin.com,MICROSOFT-CORP-MSN-AS-BLOCK,AS8068 MICROSOFT-CORP-MSN-AS-BLOCK
2,http://tatar-congress.org/ru/yanalyklar/video-...,2021-01-13 11:11:19,014c731f5479c72d619fb4f57e7cc85bc17d6c78ea4221...,0,NaN,185.75.47.6,AS62222,tatar-congress.org,tatar-congress.org,QuickSoft LLC,AS62222 QuickSoft LLC
3,https://arsitios.com/zonafrancatv/2020/08/17/v...,2021-01-13 11:11:20,d6e4b90ad2f9285c6c9af41d64b5ed68fe1884bf73ff30...,1,Kaspersky:phishing site,172.67.177.187,AS13335,arsitios.com,arsitios.com,CLOUDFLARENET,AS13335 CLOUDFLARENET
4,http://guiaglobal.com.br/2020/08/17/imunologis...,2021-01-13 11:11:21,10ac268d65affc488c0ca1abc26efc67e39680f66075e9...,0,NaN,191.252.123.29,AS27715,guiaglobal.com.br,guiaglobal.com.br,Locaweb Servicos de Internet S/A,AS27715 Locaweb Servicos de Internet S/A


In [12]:
vt_df["org"].unique()

array(['AMAZON-02', 'MICROSOFT-CORP-MSN-AS-BLOCK', 'QuickSoft LLC',
       'CLOUDFLARENET', 'Locaweb Servicos de Internet S/A', 'FASTLY',
       'iTools JSC', 'ACOD JSC', 'AUTOMATTIC', None, '1&1 Ionos Se',
       'Dattatec.com', 'INTERNET-ARCHIVE', 'GOOGLE', 'AMAZON-AES',
       "Seflow S.N.C. Di Marco Brame' & C.", 'DREAMLINE CO.',
       'Kakao Corp', 'OVH SAS', 'LeaseWeb Netherlands B.V.',
       'ASMALLORANGE1', 'UNIFIEDLAYER-AS-1', 'Hetzner Online GmbH',
       'Contabo GmbH', 'AS-26496-GO-DADDY-COM-LLC',
       'Universo Online S.A.', 'PT Media Televisi Indonesia', 'HAWKHOST',
       'LEASEWEB-USA-WDC', 'DIGITALOCEAN-ASN', 'SUCURI-SEC',
       'Akamai International B.V.', 'ScaleUp Technologies GmbH & Co. KG',
       'Digmia s.r.o.', 'home.pl S.A.', 'AKAMAI-AS',
       'GTT Communications Inc.', 'TOTAL-SERVER-SOLUTIONS',
       'Rambler Internet Holding LLC', 'LEVEL3',
       'Hostinger International Limited', 'SOFTLAYER',
       'Equinix Jpapan Enterprise K.K.', 'AS-30083-GO-DAD

In [13]:
org_g = vt_df.groupby("org").size().reset_index(name = "count").sort_values(by = "count", ascending = False)
org_g.head(50)

,org,count
16,CLOUDFLARENET,75
50,MICROSOFT-CORP-MSN-AS-BLOCK,44
29,GOOGLE-PRIVATE-CLOUD,25
4,AMAZON-02,23
3,AKAMAI-AS,16
55,OVH SAS,15
28,GOOGLE,13
0,1&1 Ionos Se,12
27,FASTLY,12
12,Akamai International B.V.,11


In [14]:
vt_df["is_ipv4"] = vt_df["ip"].apply(is_ipv4)

In [15]:
vt_df[vt_df["is_ipv4"] == False].shape

(8, 12)

In [16]:
ips = vt_df[vt_df["is_ipv4"] == True]["ip"].drop_duplicates(keep = "first")
ips.to_csv("../data/mal_ips", header = False, index = False)

In [17]:
ip_features = pd.read_csv("../data/mal_ips.pdns", header = None, sep = ",")
ip_features.columns = ["ip", "firstseen", "lastseen", "#queries", "#apexes"]

In [18]:
ip_features.head()

,ip,firstseen,lastseen,#queries,#apexes
0,13.107.42.14,2018-04-25 17:07:16,2021-01-12 19:34:56,2273869086,6
1,185.75.47.6,2020-03-02 11:01:20,2021-01-12 18:26:10,5709,8
2,172.67.177.187,2020-05-27 21:25:49,2021-01-12 23:54:51,3830787,1128
3,191.252.123.29,2020-02-18 18:34:18,2021-01-12 19:03:29,13733,5
4,104.31.78.53,2015-02-13 02:03:02,2021-01-12 23:58:44,2648277,1050


In [19]:
ip_features[ip_features["#apexes"] < 10].shape

(92, 5)

In [20]:
ip_features["ip"] = ip_features["ip"].str.strip()
vt_df["ip"] = vt_df["ip"].str.strip()
ip_features2 = pd.merge(vt_df[vt_df["is_ipv4"] == True][["url", "ip", "asn", "org"]],
                       ip_features[["ip", "#queries", "#apexes"]], on = "ip", how = "inner")

In [21]:
ip_features2.head()

,url,ip,asn,org,#queries,#apexes
0,https://www.linkedin.com/feed/update/urn:li:sh...,13.107.42.14,AS8068,MICROSOFT-CORP-MSN-AS-BLOCK,2273869086,6
1,https://www.linkedin.com/feed/update/urn:li:sh...,13.107.42.14,AS8068,MICROSOFT-CORP-MSN-AS-BLOCK,2273869086,6
2,https://www.linkedin.com/,13.107.42.14,AS8068,MICROSOFT-CORP-MSN-AS-BLOCK,2273869086,6
3,https://www.linkedin.com/feed/update/urn:li:sh...,13.107.42.14,AS8068,MICROSOFT-CORP-MSN-AS-BLOCK,2273869086,6
4,https://www.linkedin.com/feed/update/urn:li:sh...,13.107.42.14,AS8068,MICROSOFT-CORP-MSN-AS-BLOCK,2273869086,6


In [22]:
ip_features2.to_csv("../data/ip_features.csv", header = True, index = False)

In [23]:
ip_features2[ip_features2["#apexes"] >= 100].shape

(136, 6)

In [24]:
ip_features2.shape

(406, 6)

In [25]:
136/406

0.33497536945812806

In [26]:
sys.path.append("../../Common/utils")
from lex_features import get_features_urls
urls = set(ip_features2["url"].unique())
lex_features = get_features_urls(urls)

In [27]:
lf_df = pd.DataFrame().from_records(lex_features)
lf_df.head()

,url,protocol,num_queries,path_depth,domain,suspicious_tld,length,entropy,fake_tld,brand,pop_keywords,similar,num_subdomains
0,https://www.pronews.gr/ygeia/913333_emvolio-as...,https,0,3,www.pronews.gr,0,14,0.368776,0,0,0,0,1
1,https://linktr.ee/smartmedia%20https:/gigazine...,https,0,6,linktr.ee,0,9,0.368463,0,0,0,0,0
2,https://www.linkedin.com/,https,0,2,www.linkedin.com,0,16,0.415977,0,0,0,1,1
3,https://m.facebook.com/112639733519315/posts/3...,https,0,5,m.facebook.com,0,14,0.379894,0,1,0,1,1
4,https://piemovies.cc/watch/sputnik-2020,https,0,3,piemovies.cc,1,12,0.385620,0,0,0,0,0


In [30]:
all_features = pd.merge(lf_df, ip_features2, on = "url", how = "inner")

In [38]:
#all_features = all_features.drop('domain', 1)
all_features["subnet24"] = all_features["ip"].apply(get_subnet24)
all_features = all_features.drop('ip', 1)
all_features.to_csv("../data/all_features.csv", header = True, index = False)

In [28]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, MinMaxScaler



def transform_cols(df, cols):
    for col in cols:
        df[col] = LabelEncoder().fit_transform(df[col])
    return df



In [29]:
from urllib.parse import urlparse
x = urlparse("https://hello.com/f1/x1?a=5&b=3")
x.scheme

'https'